<a href="https://colab.research.google.com/github/Rasa16/Face_recognition/blob/master/face_recognition_own_opencv_encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join(["/content/drive/My Drive/Face_recognition_own/face_recognition_own_opencv/opencv-face-recognition/face_detection_model/", "deploy.prototxt"])
modelPath = os.path.sep.join(["/content/drive/My Drive/Face_recognition_own/face_recognition_own_opencv/opencv-face-recognition/face_detection_model/",
	"res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch("/content/drive/My Drive/Face_recognition_own/face_recognition_own_opencv/opencv-face-recognition/openface_nn4.small2.v1.t7")

[INFO] loading face detector...
[INFO] loading face recognizer...


In [ ]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images("/content/drive/My Drive/Face_recognition_own/My family"))

# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

[INFO] quantifying faces...


In [ ]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()
 
	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
				(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1



[INFO] processing image 1/136
[INFO] processing image 2/136
[INFO] processing image 3/136
[INFO] processing image 4/136
[INFO] processing image 5/136
[INFO] processing image 6/136
[INFO] processing image 7/136
[INFO] processing image 8/136
[INFO] processing image 9/136
[INFO] processing image 10/136
[INFO] processing image 11/136
[INFO] processing image 12/136
[INFO] processing image 13/136
[INFO] processing image 14/136
[INFO] processing image 15/136
[INFO] processing image 16/136
[INFO] processing image 17/136
[INFO] processing image 18/136
[INFO] processing image 19/136
[INFO] processing image 20/136
[INFO] processing image 21/136
[INFO] processing image 22/136
[INFO] processing image 23/136
[INFO] processing image 24/136
[INFO] processing image 25/136
[INFO] processing image 26/136
[INFO] processing image 27/136
[INFO] processing image 28/136
[INFO] processing image 29/136
[INFO] processing image 30/136
[INFO] processing image 31/136
[INFO] processing image 32/136
[INFO] processing

In [ ]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("/content/drive/My Drive/Face_recognition_own/face_recognition_own_opencv/encodings_own_opencv.pickle", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 135 encodings...
